# Fast Fuzzy Matching 2.0
Using cosine similarity, we are able to increase the speed of fuzzy matching by up to 40X for large scale data sets of 100 Million or more comparisons.

# Install Python Libraries

In [1]:
#!pip install rapidfuzz #!conda install -c conda-forge rapidfuzz --yes
#!pip install cython
#!pip install sparse_dot_topn

In [2]:
# Load libraries
import re
import time
import numpy as np
import pandas as pd
import operator
import rapidfuzz
from rapidfuzz import process, fuzz, utils
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct

# Import Data

In [3]:
nasdaq = pd.read_csv('data/nasdaq.csv')
sp500 = pd.read_csv('data/s_and_p_500.csv')
nyse = pd.read_csv('data/nyse.csv')
other = pd.read_csv('data/other.csv')
sec = pd.read_csv('data/sec_edgar_company_info.csv')

nasdaq['Stock Exchange'] = 'Nasdaq'
sp500['Stock Exchange'] = 'S&P 500'
nyse['Stock Exchange'] = 'NYSE'
other['Stock Exchange'] = 'Other'

stocks = pd.concat([nasdaq,sp500,nyse,other])
stocks = stocks.drop_duplicates(subset = 'Symbol')

In [4]:
stocks.head(2)

,Symbol,Company Name,Stock Exchange
0,AAIT,iShares MSCI All Country Asia Information Tech...,Nasdaq
1,AAL,"American Airlines Group, Inc.",Nasdaq


In [5]:
sec.head(2)

,Company Name,Company CIK Key
0,!J INC,1438823
1,"#1 A LIFESAFER HOLDINGS, INC.",1509607


# Pre-processing
Pre-processing strings in both datasets by replacing non-alphanumeric characters with whitespace, making strings lowercase, and stripping added whitespace.

In [6]:
%%time
stocks['Company Name Clean'] = stocks['Company Name'].apply(lambda x: utils.default_process(x)).str.replace('[^A-z0-9 ]+', '').str.replace(' +', ' ')
sec['Company Name Clean'] = sec['Company Name'].astype(str).apply(lambda x: utils.default_process(x)).str.replace('[^A-z0-9 ]+', '').str.replace(' +', ' ')

CPU times: user 2.23 s, sys: 58.9 ms, total: 2.29 s
Wall time: 2.29 s


## Full Match
Once the pre-processing is complete, we want to first perform a full match on the cleaned company names to reduce the complexity of the fuzzy matching on remaining names.

In [7]:
stocks_full_match = stocks.merge(sec, how = 'inner', on = 'Company Name Clean', suffixes = (' Set A', ' Set B'))\

print(len(stocks_full_match), 'full matches out of', len(stocks),
      '({:.1%})'.format(len(stocks_full_match)/len(stocks)))

2628 full matches out of 8190 (32.1%)


In [8]:
# Separate unmatched stocks and SEC companies to use for fuzzy matching
stocks_not_matched = stocks[~stocks['Company Name Clean'].isin(stocks_full_match['Company Name Clean'])]
sec_not_matched = sec[~sec['Company Name Clean'].isin(stocks_full_match['Company Name Clean'])]

print(len(stocks_not_matched), 'stocks not matched out of', len(stocks),
      '({:.1%})'.format(len(stocks_not_matched)/len(stocks)))
print(len(sec_not_matched), 'SEC companies not matched out of', len(stocks),
      '({:.1%})'.format(len(sec_not_matched)/len(sec)))

5904 stocks not matched out of 8190 (72.1%)
660559 SEC companies not matched out of 8190 (99.6%)


# Fast Fuzzy Match
Extract the best match for each stock against the 660k SEC company names and use a score cutoff of 90% match to join the 2 datasets on approximate company name

In [9]:
# Load libraries
#import re
#import time
#import operator
#import numpy as np
#from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
#from scipy.sparse import csr_matrix
#import pandas as pd

#import sparse_dot_topn.sparse_dot_topn as ct

# A class for matching one list of strings to another
class StringMatch():
    
    def __init__(self, source_names, target_names):
        self.source_names = source_names
        self.target_names = target_names
        self.ct_vect      = None
        self.tfidf_vect   = None
        self.vocab        = None
        self.sprse_mtx    = None
        
        
    def tokenize(self, analyzer='char_wb', n=3):
        '''
        Tokenizes the list of strings, based on the selected analyzer
        :param str analyzer: Type of analyzer ('char_wb', 'word'). Default is trigram
        :param str n: If using n-gram analyzer, the gram length
        '''
        # Create initial count vectorizer & fit it on both lists to get vocab
        self.ct_vect = CountVectorizer(analyzer=analyzer, ngram_range=(n, n))
        self.vocab   = self.ct_vect.fit(self.source_names + self.target_names).vocabulary_
        
        # Create tf-idf vectorizer
        self.tfidf_vect  = TfidfVectorizer(vocabulary=self.vocab, analyzer=analyzer, ngram_range=(n, n))
        
        
    def match(self, ntop=1, lower_bound=0, output_fmt='df'):
        '''
        Main match function. Default settings return only the top candidate for every source string.
        
        :param int ntop: The number of top-n candidates that should be returned
        :param float lower_bound: The lower-bound threshold for keeping a candidate, between 0-1.
                                   Default set to 0, so consider all canidates
        :param str output_fmt: The output format. Either dataframe ('df') or dict ('dict')
        '''
        self._awesome_cossim_top(ntop, lower_bound)
        
        if output_fmt == 'df':
            match_output = self._make_matchdf()
        elif output_fmt == 'dict':
            match_output = self._make_matchdict()
            
        return match_output
        
        
    def _awesome_cossim_top(self, ntop, lower_bound):
        ''' https://gist.github.com/ymwdalex/5c363ddc1af447a9ff0b58ba14828fd6#file-awesome_sparse_dot_top-py '''
        # To CSR Matrix, if needed
        A = self.tfidf_vect.fit_transform(self.source_names).tocsr()
        B = self.tfidf_vect.fit_transform(self.target_names).transpose().tocsr()
        M, _ = A.shape
        _, N = B.shape

        idx_dtype = np.int32

        nnz_max = M * ntop

        indptr = np.zeros(M+1, dtype=idx_dtype)
        indices = np.zeros(nnz_max, dtype=idx_dtype)
        data = np.zeros(nnz_max, dtype=A.dtype)

        ct.sparse_dot_topn(
            M, N, np.asarray(A.indptr, dtype=idx_dtype),
            np.asarray(A.indices, dtype=idx_dtype),
            A.data,
            np.asarray(B.indptr, dtype=idx_dtype),
            np.asarray(B.indices, dtype=idx_dtype),
            B.data,
            ntop,
            lower_bound,
            indptr, indices, data)

        self.sprse_mtx = csr_matrix((data,indices,indptr), shape=(M,N))
    
    
    def _make_matchdf(self):
        ''' Build dataframe for result return '''
        # CSR matrix -> COO matrix
        cx = self.sprse_mtx.tocoo()

        # COO matrix to list of tuples
        match_list = []
        for row,col,val in zip(cx.row, cx.col, cx.data):
            match_list.append((row, self.source_names[row], col, self.target_names[col], val))

        # List of tuples to dataframe
        colnames = ['Set_A_Index', 'Set_A', 'Set_B_Index', 'Set_B', 'Match Ratio']
        match_df = pd.DataFrame(match_list, columns=colnames)
        match_df['Match Ratio'] = match_df['Match Ratio'] * 100

        return match_df

    
    def _make_matchdict(self):
        ''' Build dictionary for result return '''
        # CSR matrix -> COO matrix
        cx = self.sprse_mtx.tocoo()

        # dict value should be tuple of values
        match_dict = {}
        for row,col,val in zip(cx.row, cx.col, cx.data):
            if match_dict.get(row):
                match_dict[row].append((col,val))
            else:
                match_dict[row] = [(col, val)]

        return match_dict   

def fast_fuzzy_match(df1,col1,df2,col2,num_matches=1,cutoff=0):
    titlematch = StringMatch(df1[col1].tolist(), df2[col2].tolist())
    titlematch.tokenize()
    nlp_df = titlematch.match(ntop=num_matches,lower_bound=(cutoff/100))
    return nlp_df

In [10]:
%%time
# Run the fuzzy match logic on the SEC and company data
score_cutoff = 80
set_a = stocks_not_matched
col_a = 'Company Name Clean'
set_b = sec_not_matched
col_b = 'Company Name Clean'

df = fast_fuzzy_match(set_a,col_a, set_b, col_b,cutoff=score_cutoff)

# Merge results
df = stocks_not_matched.merge(df, how = 'left', left_on= 'Company Name Clean', right_on='Set_A').drop_duplicates(subset='Symbol').merge(sec_not_matched, how = 'left', left_on = 'Set_B', right_on = 'Company Name Clean', suffixes = (' Set A', ' Set B')).drop_duplicates(subset='Symbol')

CPU times: user 56.2 s, sys: 488 ms, total: 56.7 s
Wall time: 56.6 s


In [11]:
df.head(5)

,Symbol,Company Name Set A,Stock Exchange,Company Name Clean Set A,Set_A_Index,Set_A,Set_B_Index,Set_B,Match Ratio,Company Name Set B,Company CIK Key,Company Name Clean Set B
0,AAIT,iShares MSCI All Country Asia Information Tech...,Nasdaq,ishares msci all country asia information tech...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AAME,Atlantic American Corporation,Nasdaq,atlantic american corporation,1.0,atlantic american corporation,44149.0,atlantic american corp,91.545702,ATLANTIC AMERICAN CORP,8177.0,atlantic american corp
2,AAWW,Atlas Air Worldwide Holdings,Nasdaq,atlas air worldwide holdings,2.0,atlas air worldwide holdings,44483.0,atlas air worldwide holdings inc,98.599175,ATLAS AIR WORLDWIDE HOLDINGS INC,1135185.0,atlas air worldwide holdings inc
3,AAXJ,iShares MSCI All Country Asia ex Japan Index Fund,Nasdaq,ishares msci all country asia ex japan index fund,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABCO,The Advisory Board Company,Nasdaq,the advisory board company,4.0,the advisory board company,14935.0,advisory board co,85.368878,ADVISORY BOARD CO,1157377.0,advisory board co


In [12]:
print(len(df.dropna(subset = ['Company CIK Key'])), 'stocks matched out of', len(stocks),
      '({:.1%})'.format(len(df.dropna(subset = ['Company CIK Key']))/len(stocks)))

3094 stocks matched out of 8190 (37.8%)
